In [1]:
import lib.facebook_scraper as fs
import lib.utils as utils
import pandas as pd
import numpy as np
import os
import time

In [2]:
post_ids_df = pd.read_csv('data/vebay69/clean/post_ids.csv')
post_ids_df.head()

,post_id
0,685638947004646
1,685577330344141
2,685111467057394
3,684686633766544
4,684603707108170


In [3]:
fs.set_cookies('./cookies/cookies_1.txt')
# fs.enable_logging()

In [4]:
NUMBER_POST = 10
post_list = []
reactor_list = []
total_reactor_real = 0
index_post_id_file_path = os.path.join('data','vebay69','url','i_post_id_react.txt')

In [5]:
index_post_id = utils.read_index_file(file_path=index_post_id_file_path)
print(index_post_id)
posts_ids = utils.get_post_id_list(df=post_ids_df, start=index_post_id, length=NUMBER_POST)
posts_ids

[Errno 2] No such file or directory: 'data/vebay69/url/i_post_id_react.txt'
Auto create file: ./data/vebay69/url/i_post_id_react.txt
0


[685638947004646,
 685577330344141,
 685111467057394,
 684686633766544,
 684603707108170,
 684191557149385,
 683955240506350,
 682996903935517,
 683571943878013,
 683467500555124]

In [6]:
def extract_reactor(reactors, post_id):
    cnt_reactor = 0
    global reactor_list
    for reactor in reactors:
        reactor['post_id'] = post_id
        reactor_list.append(reactor)
        cnt_reactor += 1
        time.sleep(np.random.uniform(0.5, 2.5))
    return cnt_reactor

In [7]:
try:
    for post in fs.get_posts(
        post_urls=posts_ids,
        options={
            "allow_extra_requests": True,
            "reactors": True,
            "reactors": "generator",
            "reactions": True
        },
        timeout=120,
    ):
        print(post)
        post_list.append(post)
        num_sharer = extract_reactor(post['reactors'], post['original_request_url'])
        print(f'Extracted {num_sharer}/{post["reaction_count"]} reactors')
        total_reactor_real += post['reaction_count']
        utils.sleep(np.random.randint(60, 180))
        print()
except fs.exceptions.TemporarilyBanned:
    print("TEMPORARILY BANNED")
except fs.exceptions.AccountDisabled:
    print("ACCOUNT DISABLED")
except Exception as e:
    print(e)

/home/ad/Workspace/data-processing/analyze-facebook-page/lib/facebook_scraper/facebook_scraper.py:912: UserWarning: Facebook says 'Unsupported Browser'
  warnings.warn(f"Facebook says 'Unsupported Browser'")


{'original_request_url': 685638947004646, 'post_url': 'https://facebook.com/story.php?story_fbid=pfbid02LrmevXfzAdCYohYbPeEdz2Tg2zpdpZRsaRHZC3HrpK8tXRTcmHgVGZrBhiTh6BNGl&id=100066756416622', 'post_id': 'pfbid02LrmevXfzAdCYohYbPeEdz2Tg2zpdpZRsaRHZC3HrpK8tXRTcmHgVGZrBhiTh6BNGl', 'text': 'Mùa đông chưa vậy\n#Panda', 'post_text': 'Mùa đông chưa vậy\n#Panda', 'shared_text': '', 'original_text': None, 'time': datetime.datetime(2023, 11, 20, 12, 28, 2), 'timestamp': 1700458082, 'image': 'https://scontent.fhan14-1.fna.fbcdn.net/v/t39.30808-6/404513275_685638733671334_3197628116849525623_n.jpg?stp=cp0_dst-jpg_e15_fr_q65&_nc_cat=107&ccb=1-7&_nc_sid=5f2048&efg=eyJpIjoidCJ9&_nc_eui2=AeEJy8ASnI-drO3c0kD34jTBkkYLACx62iSSRgsALHraJKpo5yjhr5Vx15rQKDFkmuC_stIZOAgtmKNuJp56-GYS&_nc_ohc=E9eLc0YDr0QAX-CKk4C&_nc_ht=scontent.fhan14-1.fna&oh=00_AfDZNMHaq-PtufLwhPLvMtsMUd_y14W2dbYCt_X_I8P7cA&oe=6565C85D&manual_redirect=1', 'image_lowquality': 'https://scontent.fhan14-1.fna.fbcdn.net/v/t39.30808-6/404513275_6856

/home/ad/Workspace/data-processing/analyze-facebook-page/lib/facebook_scraper/facebook_scraper.py:908: UserWarning: Facebook served mbasic/noscript content unexpectedly on https://m.facebook.com/ufi/reaction/profile/browser/fetch/?ft_ent_identifier=pfbid02LrmevXfzAdCYohYbPeEdz2Tg2zpdpZRsaRHZC3HrpK8tXRTcmHgVGZrBhiTh6BNGl&limit=50&shown_ids=1750938852%2C1739815537%2C1695082608%2C1691571596%2C1685873757%2C1671182136%2C1669660654%2C1659470191%2C1658691304%2C1644385313%2C1581493289%2C1566065791%2C1551560089%2C1492264905%2C1367927905%2C1319443675%2C1314103314%2C1310614938%2C1292667615%2C1273149500%2C1248549442%2C1240938016%2C1055541126%2C1040331288%2C1021964875%2C790538889%2C730666262%2C694449589%2C683560351%2C669272096%2C643000798%2C604385192%2C589541798%2C501219366%2C72701045%2C100000092734793%2C100000089819280%2C100000089624397%2C100000082303909%2C100000080318023%2C100000073703088%2C100000073110942%2C100000070123283%2C100000069791685%2C100000065276458%2C100000062857799%2C100000062531574%2

Extracted 2780/2789 reactors
	Sleep 101 seconds


KeyboardInterrupt: 

In [8]:
print(NUMBER_POST)
print(f"Số posts đã crawl: {len(post_list)}")
print(f"Số sharers đã crawl: {len(reactor_list)}/{total_reactor_real}")

10
Số posts đã crawl: 1
Số sharers đã crawl: 2780/2789


In [9]:
if reactor_list:
    reactor_df = utils.save_data(
        data_list=reactor_list,
        folder_path=os.path.join('data','vebay69','raw'),
        type="reactors"
    )
utils.write_index_file(
    file_path=index_post_id_file_path,
    index=index_post_id + len(post_list)
)

Save reactors data: ./data/vebay69/raw/reactors_2023-11-25_03-16-12.csv
Save index 1 to resume: ./data/vebay69/url/i_post_id_react.txt
